In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
import pandas as pd
import time
import datetime
from datetime import timedelta
%matplotlib inline

## 獲取兩個日期間的所有日期

In [3]:
def getEveryDay(begin_date,end_date):
    date_list = []
    begin_date = datetime.datetime.strptime(begin_date, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(end_date,"%Y-%m-%d")
    while begin_date <= end_date:
        date_str = begin_date.strftime("%Y-%m-%d")
        date_list.append(date_str)
        begin_date += datetime.timedelta(days=1)
        
    return date_list

In [30]:
df = pd.read_table("stat_v18.txt")
df = df.drop(['PMfstrS','PMfstrL','PMfstrA','PMfu'],axis = 1)
df = df[df['tau']!=0]
df = df[df['stationID']!='EPA101']
df.head()

/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


,year,month,day,hour,expdate,tau,ver,stationID,calPMf
1848,2019,5,2,0,2019-05-01,1,18,EPA001,25.378662
1849,2019,5,2,0,2019-05-01,1,18,EPA002,26.762809
1850,2019,5,2,0,2019-05-01,1,18,EPA003,28.312006
1851,2019,5,2,0,2019-05-01,1,18,EPA004,23.493353
1852,2019,5,2,0,2019-05-01,1,18,EPA005,16.208615


In [31]:
pd.notnull(df[df['stationID']=='EPA101']['calPMf'])

Series([], Name: calPMf, dtype: bool)

In [32]:
df['year'] = df['year'].astype(str)
df['month'] = df['month'].astype(str)
df['day'] = df['day'].astype(str)
df['hour'] = df['hour'].astype(str)
df['dateTime'] = df['year'] + '-' + df['month'].str.zfill(2) + '-'+ df['day'].str.zfill(2) + ' '+df['hour'].str.zfill(2) +':'+ '00:00 '
#df['expdate'] = df['expdate'].str.split('/').str.get(0) + '-' + df['expdate'].str.split('/').str.get(1).str.zfill(2) + '-'+ df['expdate'].str.split('/').str.get(2).str.zfill(2)
df['dateTime'] = pd.to_datetime(df['dateTime'], errors='coerce')
df.head()

,year,month,day,hour,expdate,tau,ver,stationID,calPMf,dateTime
1848,2019,5,2,0,2019-05-01,1,18,EPA001,25.378662,2019-05-02
1849,2019,5,2,0,2019-05-01,1,18,EPA002,26.762809,2019-05-02
1850,2019,5,2,0,2019-05-01,1,18,EPA003,28.312006,2019-05-02
1851,2019,5,2,0,2019-05-01,1,18,EPA004,23.493353,2019-05-02
1852,2019,5,2,0,2019-05-01,1,18,EPA005,16.208615,2019-05-02


In [7]:
df_add['dateTime'] = list(str(pd.to_datetime(date, errors='coerce') + timedelta(days=1)))*24 + \
                             list(str(pd.to_datetime(date, errors='coerce') + timedelta(days=2)))*24 + \
                            list(str(pd.to_datetime(date, errors='coerce') + timedelta(days=3)))*24 + \
                             list(str(pd.to_datetime(date, errors='coerce') + timedelta(days=4)))*24
df_add

NameError: name 'date' is not defined

In [33]:
stations = df['stationID'].unique()
all_expdate = getEveryDay('2019-05-01','2019-08-27')

for name in stations:
    unnormal = list(df[df['stationID'] == name]['expdate'])
    dates = list(set(all_expdate).difference(set(unnormal)))
    print(name)
    for date in dates:

        #str = int(date.split("/")[2])
        df_add = df[:  96].copy()
        df_add['expdate'] = date
        df_add['hour'] = list(np.arange(24))*4
        df_add['stationID'] = name
        df_add['tau'] = np.arange(96) + 1
        df_add['dateTime'] = pd.to_datetime(date, errors='coerce') + timedelta(days=1)
        #df_add['dateTime'] = pd.to_datetime(df_add['dateTime'], errors='coerce')
        df_add['month'] = df_add['dateTime'].dt.month
        df_add['day'] = df_add['dateTime'].dt.day
        df_add['calPMf'] = np.nan
        df_add['ver'] = [18]* 96

        df = pd.concat([df, df_add])

    df.sort_values(by=['expdate', 'tau','stationID'], inplace = True)
    if sum(pd.value_counts(df[df['stationID']==name]['expdate'])!=96)>0:
        count = pd.value_counts(df[df['stationID']==name ]['expdate']).values
        index = pd.value_counts(df[df['stationID']==name ]['expdate']).index
        for expdate in index[count!=96]:
            all_hour = np.arange(96) + 1
            unnormal = df[df['stationID']==name ][df['expdate']==expdate]['tau']
            taus = list(set(all_hour).difference(set(unnormal)))
            num = len(hours)
            #timestamp = int(time.mktime(time.strptime(expdate, "%Y-%m-%d"))) ################

            df_add = df[:  len(taus)].copy()
            df_add['expdate'] = expdate  
            df_add['hour'] = taus % 24 
            df_add['stationID'] = name
            df_add['tau'] = taus
            df_add['month'] = np.nan ###########
            df_add['day'] = np.nan
            df_add['calPMf'] = np.nan
            df_add['ver'] = [17] * num
            timeArray = time.strptime(expdate + ' 00', "%Y-%m-%d %H")
            #df_add['dateTime'] =  timestamp +86400 + taus * 3600 ######################
            df = pd.concat([df, df_add])

EPA001
EPA002
EPA003
EPA004
EPA005
EPA006
EPA007
EPA008
EPA009
EPA010
EPA011
EPA012
EPA013
EPA014
EPA015
EPA016
EPA017
EPA018
EPA019
EPA020
EPA021
EPA022
EPA023
EPA024
EPA025
EPA026
EPA027
EPA028
EPA029
EPA030
EPA031
EPA032
EPA033
EPA034
EPA035
EPA036
EPA037
EPA038
EPA039
EPA040
EPA041
EPA042
EPA043
EPA044
EPA045
EPA046
EPA047
EPA048
EPA049
EPA050
EPA051
EPA052
EPA053
EPA054
EPA056
EPA057
EPA058
EPA059
EPA060
EPA061
EPA062
EPA063
EPA064
EPA065
EPA066
EPA067
EPA068
EPA069
EPA070
EPA071
EPA072
EPA080
EPA084
EPA102
EPA104
EPA110


In [141]:
#df.to_csv('pred_v18.csv')

In [34]:
df_pm25 = pd.DataFrame(np.array(df['calPMf']).reshape([-1, 96*76]))
df_pm25.index = pd.to_datetime(df['expdate'].unique(), errors='coerce') + timedelta(days=1)
df_pm25.head()

,0,1,2,3,4,5,6,7,8,9,...,7286,7287,7288,7289,7290,7291,7292,7293,7294,7295
2019-05-02,25.378662,26.762809,28.312006,23.493353,16.208615,23.114860,23.876440,35.129498,27.616800,23.148564,...,16.824983,12.454709,8.439036,16.809657,3.155507,1.595888,0.287451,17.073544,0.621773,2.503552
2019-05-03,22.449501,22.553379,19.701610,18.562927,11.574634,23.212830,31.011216,29.423361,28.683004,25.265536,...,2.253691,13.458723,4.180285,25.224034,8.433666,1.709137,0.342138,58.602387,1.133107,6.348839
2019-05-04,15.152556,15.216946,16.225082,15.661366,15.491049,17.479730,19.955730,20.260817,20.356940,16.514298,...,6.347907,15.263381,5.190069,38.129214,8.861641,4.290921,0.478406,61.103512,8.076694,5.772860
2019-05-05,14.915360,17.118671,15.830101,17.272750,19.017453,20.192850,15.744980,22.260971,17.311502,17.577911,...,15.756956,11.659058,10.577075,13.918918,5.631461,1.661173,1.575462,20.400735,7.972200,14.445440
2019-05-06,20.169198,20.062555,13.534532,16.130718,14.725401,15.050772,15.686775,19.713494,20.097872,21.020320,...,9.668982,5.287912,4.084679,19.218267,10.815857,2.114089,0.240413,55.352082,15.559785,9.011922


In [35]:
df_pm25.shape

(119, 7296)

In [186]:
df_pm25.to_csv('pred_v17_change.csv')

## -------------------------------------------------

In [36]:
no_nan_df = df_pm25.dropna(axis=0, how='all')
no_nan_df.head()

,0,1,2,3,4,5,6,7,8,9,...,7286,7287,7288,7289,7290,7291,7292,7293,7294,7295
2019-05-02,25.378662,26.762809,28.312006,23.493353,16.208615,23.114860,23.876440,35.129498,27.616800,23.148564,...,16.824983,12.454709,8.439036,16.809657,3.155507,1.595888,0.287451,17.073544,0.621773,2.503552
2019-05-03,22.449501,22.553379,19.701610,18.562927,11.574634,23.212830,31.011216,29.423361,28.683004,25.265536,...,2.253691,13.458723,4.180285,25.224034,8.433666,1.709137,0.342138,58.602387,1.133107,6.348839
2019-05-04,15.152556,15.216946,16.225082,15.661366,15.491049,17.479730,19.955730,20.260817,20.356940,16.514298,...,6.347907,15.263381,5.190069,38.129214,8.861641,4.290921,0.478406,61.103512,8.076694,5.772860
2019-05-05,14.915360,17.118671,15.830101,17.272750,19.017453,20.192850,15.744980,22.260971,17.311502,17.577911,...,15.756956,11.659058,10.577075,13.918918,5.631461,1.661173,1.575462,20.400735,7.972200,14.445440
2019-05-06,20.169198,20.062555,13.534532,16.130718,14.725401,15.050772,15.686775,19.713494,20.097872,21.020320,...,9.668982,5.287912,4.084679,19.218267,10.815857,2.114089,0.240413,55.352082,15.559785,9.011922
2019-05-07,2.671120,19.065295,11.045278,15.858298,7.119115,14.933830,16.434140,22.944383,21.296416,11.993011,...,14.878386,2.921710,5.848060,33.528150,8.492652,0.954279,0.248653,43.434278,1.409147,3.812406
2019-05-08,12.008221,13.797399,10.839516,16.179682,16.902564,18.532970,18.405900,18.349056,17.996240,10.517890,...,11.354709,32.790200,5.253278,55.485967,3.592080,1.540678,0.081880,154.920089,0.738440,7.617957
2019-05-09,13.321711,11.622199,8.176604,6.942328,10.265822,18.744730,9.877997,17.956333,10.395360,8.754540,...,11.937148,42.625868,11.933568,67.268092,16.879745,2.530862,0.189464,12.129671,5.239619,6.821362
2019-05-10,1.933406,4.448012,1.451286,3.795055,4.300208,6.981692,11.343529,8.562552,7.815954,4.277438,...,15.316927,4.454513,6.555558,86.911577,8.862767,2.843997,0.070353,30.569538,2.165869,9.064067
2019-05-11,13.779223,14.141935,10.239069,12.601817,15.225530,17.410479,20.424940,14.646621,18.696180,16.679609,...,9.542719,12.419229,8.117783,26.293106,7.289289,2.331160,0.453364,170.876333,11.899172,6.210395


In [37]:
no_nan_df.shape

(117, 7296)

In [38]:
pred_v17 = pd.read_csv('after sinkr pred_v17.csv', header = None)
pred_v18 = pd.read_csv('after sinkr pred_v18.csv', header = None)
pred_v17 = pred_v17[1:]
pred_v18 = pred_v18[1:]
print("pred_v17.shape = ", pred_v17.shape)
print("pred_v18.shape = ", pred_v18.shape)

pred_v17.shape =  (120, 7296)
pred_v18.shape =  (119, 7296)


In [27]:
pred_v17.index = pd.to_datetime(df['expdate'].unique(), errors='coerce') + timedelta(days=1)
pred_v17.head()

,0,1,2,3,4,5,6,7,8,9,...,7286,7287,7288,7289,7290,7291,7292,7293,7294,7295
2019-01-02,8.802175,7.717532,8.9382284,6.06552,10.623307,9.001223,12.692663,9.44195,9.325263,3.1252,...,10.842381,12.073497,5.050655,18.26207,30.7273,6.786378,0.3042748,1.75677164,4.1410284,5.1542614
2019-01-03,4.778693,7.499562,6.9192015,6.120055,6.034465,6.913505,9.127993,8.908815,11.520565,3.789323,...,8.42830482,60.66407611,3.250508,12.37266001,29.78721,2.984121,0.05539496,51.112013,1.2923965,1.669799
2019-01-04,5.3309734,6.409755,6.0599542,3.782637,4.648133,3.888002,5.521788,5.253996,9.324592,2.208601,...,5.08351252,21.900514,3.234257,16.45797,12.143845,2.338926,0.2054999,98.827429,3.58112,10.72594173
2019-01-05,34.417397,35.265427,40.3591559,39.291456,33.230443,32.70254,32.898895,37.464746,31.361296,36.014968,...,11.67285,67.37695,4.56446,12.340719,98.82203,7.18382,0.665758,46.575709,27.087578,5.2232912
2019-01-06,8.1570081,8.810575,8.5095347,8.004482,15.16443,13.810938,9.51487,9.795454,12.154057,7.679361,...,7.64693371,62.35009748,2.17559545,13.61775001,108.4499,2.005029,0.52793525,11.7173785,2.2373521,2.61527649


In [28]:
df_pred_v17 = pd.concat([pred_v17, no_nan_df], axis=1, join_axes=[no_nan_df.index]).iloc[:, :7296]
df_pred_v17.shape

(106, 7296)

In [29]:
df_pred_v17.to_csv('pred17_end.csv')

## -------------------------------------------------

In [39]:
pred_v18.index = pd.to_datetime(df['expdate'].unique(), errors='coerce') + timedelta(days=1)
pred_v18.head()

,0,1,2,3,4,5,6,7,8,9,...,7286,7287,7288,7289,7290,7291,7292,7293,7294,7295
2019-05-02,25.378662,26.762809,28.312006,23.493353,16.208615,23.11486,23.87644,35.129498,27.6168,23.148564,...,16.82498327,12.4547093,8.43903599,16.80965718,3.1555067,1.595888,0.2874513,17.0735442,0.62177321,2.50355221
2019-05-03,22.449501,22.553379,19.7016104,18.562927,11.574634,23.21283,31.011216,29.423361,28.683004,25.265536,...,2.25369052,13.458723,4.18028495,25.224034,8.4336662,1.709137,0.34213845,58.602387,1.1331074,6.348839
2019-05-04,15.152556,15.216946,16.225082,15.661366,15.491049,17.47973,19.95573,20.260817,20.35694,16.514298,...,6.347907,15.263381,5.190069,38.129214,8.861641,4.290921,0.47840581,61.103512,8.0766939,5.77286
2019-05-05,14.91536,17.118671,15.830101,17.27275,19.017453,20.19285,15.74498,22.260971,17.311502,17.577911,...,15.7569563,11.659058,10.57707462,13.9189177,5.631461,1.661173,1.575462,20.4007346,7.9722,14.44544
2019-05-06,20.169198,20.062555,13.534532,16.130718,14.725401,15.050772,15.686775,19.713494,20.097872,21.02032,...,9.6689822,5.287912,4.0846789,19.218267,10.815857,2.114089,0.2404131,55.352082,15.559785,9.011922


In [40]:
df_pred_v18 = pd.concat([pred_v18, no_nan_df], axis=1, join_axes=[no_nan_df.index]).iloc[:, :7296]
df_pred_v18.shape

(117, 7296)

In [41]:
df_pred_v18.to_csv('pred18_end.csv')

## -------------------------------------------------

In [85]:
observer = pd.read_csv('observer dim=7296.csv', header = None)
observer = observer.iloc[1:]
observer.index = observer[0]
observer = observer.iloc[:, 1:]
observer.head()

,1,2,3,4,5,6,7,8,9,10,...,7287,7288,7289,7290,7291,7292,7293,7294,7295,7296
0,,,,,,,,,,,,,,,,,,,,,
2019-01-01,10.0,7.0,5.0,4.0,10.0,4.0,1.0,4.0,11.0,13.0,...,78.000000,42.0,54.0,85.0,3.000000,0.0,19.0,17.0,20.0,40.0
2019-01-02,5.0,5.0,2.0,11.0,6.0,3.0,5.0,4.0,9.0,13.0,...,50.000000,9.0,41.0,30.0,0.000000,0.0,18.0,8.0,7.0,17.0
2019-01-03,6.0,5.0,1.0,10.0,8.0,1.0,6.0,6.0,11.0,7.0,...,20.000000,3.0,33.0,25.0,6.000000,0.0,16.0,14.0,14.0,14.0
2019-01-04,4.0,7.0,3.0,8.0,8.0,0.0,5.0,4.0,7.0,10.0,...,33.000000,15.0,39.0,25.0,7.000000,0.0,31.0,14.0,12.0,24.0
2019-01-05,40.0,20.0,29.0,53.0,42.0,30.0,37.0,12.0,23.0,18.0,...,22.605063,8.0,36.0,26.0,1.219811,0.0,15.0,12.0,14.0,11.0


In [86]:
pred17_end = pd.read_csv('pred17_end.csv', header = None)
pred18_end = pd.read_csv('pred18_end.csv', header = None)
pred17_end = pred17_end.iloc[1:]
pred18_end = pred18_end.iloc[1:]
pred17_end.index = pred17_end[0]
pred18_end.index = pred18_end[0]
pred17_end = pred17_end.iloc[:, 1:]
pred18_end = pred18_end.iloc[:, 1:]
print(pred17_end.shape)
print(pred18_end.shape)

(106, 7296)
(117, 7296)


In [87]:
pred = pd.concat([pred17_end, pred18_end], axis=0)
pred.shape

(223, 7296)

In [89]:
df_observer = pd.concat([observer, pred], axis=1, join_axes=[pred.index]).iloc[:, :7296]
df_observer.shape

(223, 7296)

In [90]:
df_observer.to_csv('observer_end.csv')